In [ ]:

import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from operator import itemgetter 
from PIL import Image
import os
import json
import cv2 


# DATA ANALYSIS

In [ ]:
#get annotations
direc=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations/")
anno=[]
for filn in os.listdir(direc):
    with open(direc+filn) as f:
        data=json.load(f)
        anno.append({'filename':data['FileName'],'num_Anno':data['NumOfAnno'],'Annotations':data['Annotations']})
    
    

In [ ]:
#sorting the list
anno=sorted(anno, key=itemgetter('filename'))

In [ ]:
#read train.csv
train_csv=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/train.csv"))
train_csv.head()

In [ ]:
#sorting the dataset
train_csv.sort_values(by='name',inplace=True)
train_csv.head()


In [ ]:
#length of annotaion from train_csv
print(train_csv.name.unique().shape)
#length of annotaion from train_csv
print(train_csv.classname.unique().shape)


In [ ]:
#image files
images=os.listdir(os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images"))
images.sort()
print(images[:10])
print(images[1698:1709])
print(len(images))

In [ ]:
#splitting images into train test
train=images[1698:]
test=images[:1698]

In [ ]:
cd '/kaggle/working'

In [ ]:
#getting all test objects
f= open("test.txt","w+")
for ele in test:
    f.write('/content/darknet/data/img/'+ele+'\n')
f.close()

In [ ]:
#getting height and width
direc = os.path.abspath('/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/')
dimen=[]
for filn in os.listdir(direc):
    img = Image.open(os.path.join(direc,filn))
    width=img.size[0]
    height=img.size[1]
    dimen.append([filn,width,height])


In [ ]:
#merging
dimen_df=pd.DataFrame(dimen,columns=['name','width','height'])
dimen_df.head()

In [ ]:
os.makedirs('/kaggle/working/img')

In [ ]:
#getting all train objects
f= open("train.txt","w+")
for ele in train:
    f.write('/content/darknet/data/img/'+ele+'\n')
f.close()

In [ ]:
#creating obj.names file
f= open("obj.names","w+")
for ele in labels:
    f.write(ele+'\n')
f.close()

In [ ]:
df=train_csv.merge(dimen_df, on='name')

In [ ]:
df.head()

In [ ]:
#converting x1 x2 y1 y2 -> x_center, y_center, w, h
def convert_df(df):
    
    x1 = np.array(df['x1'].values)
    x2 = np.array(df['x2'].values)
    y1 = np.array(df['y1'].values)
    y2 = np.array(df['y2'].values)
    w = np.array(df['width'].values)
    h=np.array(df['height'].values)
    dw = 1./w
    dh = 1./h
    x = (y1 + x1)/2.0
    y = (y2 + x2)/2.0
    w = y1-x1                                               
    h = y2-x2                                                 
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    data = {'x':x, 'y':y, 'w':w, 'h':h}
    df1 = pd.DataFrame(data=data)
    return pd.concat([df, df1], axis=1)

In [ ]:
df_final=convert_df(df)
df_final.head()

In [ ]:
#get labels
from sklearn.preprocessing import LabelEncoder
lb_make=LabelEncoder()
df_final["labels"] = lb_make.fit_transform(df_final["classname"])

In [ ]:
#getting list of labels with index as their categorical value
labels=list(lb_make.inverse_transform([_ for _ in range(20)]))

In [ ]:
df_final.head()

In [ ]:
#converting labels to dict
label={i:labels[i] for i in range(len(labels))}
label

In [ ]:
#shape df
print(df_final.shape)
#unique values 
print(len(df_final['name'].unique()))
#show
df_final['name'].unique()

In [ ]:
cd '/kaggle/working'

In [ ]:
#get .txt file for every image
for row in df_final.values:
    filename = str(row[0][:4])+'.txt'

    if os.path.exists(filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not

    f = open(filename,append_write)
    f.write('{0} {1} {2} {3} {4}\n'.format(row[-1],row[-5],row[-4],row[-3],row[-2]))
    f.close()

In [ ]:
name=[]
for row in df_final.values:
    name.append(row[0])

In [ ]:
name_unique=list(set(name))


In [ ]:
matchin_name=[]
for item in name_unique:
    matchin_name.append(item[:4])

In [ ]:
matchin_name_unique=list(set(matchin_name))
len(matchin_name_unique)==len(matchin_name)

In [ ]:
#get duplicate
import collections
print([item for item, count in collections.Counter(matchin_name).items() if count > 1])

### I created a obj.names file obj.data file train.txt file and .txt file for every image
##### Now i will use these files to train on darknet using yolov3 in colab which is in different file